In [2]:
from datamodules.mt_distillation_datamodule import MTDistillationDatamodule

dm = MTDistillationDatamodule(batch_size=5)
dm.setup()
batch = next(iter(dm.train_dataloader()))
batch

Using custom data configuration din0s--ccmatrix_en-ro-b5eb1c3b243f258f
Reusing dataset parquet (/home/john/.cache/huggingface/datasets/din0s___parquet/din0s--ccmatrix_en-ro-b5eb1c3b243f258f/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/1 [00:00<?, ?it/s]

Loading cached processed dataset at /home/john/.cache/huggingface/datasets/din0s___parquet/din0s--ccmatrix_en-ro-b5eb1c3b243f258f/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-246f20ddcdebc8fd.arrow
Using custom data configuration j0hngou--ccmatrix_en-fr-c75b778ef0fed3ea
Reusing dataset parquet (/home/john/.cache/huggingface/datasets/j0hngou___parquet/j0hngou--ccmatrix_en-fr-c75b778ef0fed3ea/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/1 [00:00<?, ?it/s]

Loading cached processed dataset at /home/john/.cache/huggingface/datasets/j0hngou___parquet/j0hngou--ccmatrix_en-fr-c75b778ef0fed3ea/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-7b6fd4d61823253e.arrow


Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

/home/john/anaconda3/envs/dl4nlp/lib/python3.9/site-packages/transformers/models/t5/tokenization_t5_fast.py:156: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


{'en-ro': {'source': tensor([[13959,  1566,    12,  3871,    29,    10, 15476,    13, 21268,   159,
              51,     1,     0,     0,     0,     0,     0,     0,     0,     0,
               0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
               0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
          [13959,  1566,    12,  3871,    29,    10,  2449,    33,     3, 22765,
              12,   103,    96, 23180,  1686,    68,    59,   762,     5,     1,
               0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
               0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
          [13959,  1566,    12,  3871,    29,    10,    94,    19,  1500,    12,
            1576,     5,     1,     0,     0,     0,     0,     0,     0,     0,
               0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
               0,     0,     0,     0,     0,     0,     0,     0,     0,     0]]),
  'a

In [3]:
prefix_dict = {}
prefix_dict['ro'] = "translate English to Romanian: "
prefix_dict['it'] = "translate English to Italian: "
prefix_dict['de'] = "translate English to German: "
prefix_dict['fr'] = "translate English to French: "

language_codes = ['en-ro', 'en-fr']

max_input_length = 256
max_target_length = 256


def preprocess_function(examples):
    for lcode in language_codes:
        inputs = [prefix_dict[lcode] + ex[source_lang] for ex in examples[lcode]["translation"]]
        targets = [ex[lcode] for ex in examples[lcode]["translation"]]
        model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

        # Setup the tokenizer for targets
        with tokenizer.as_target_tokenizer():
            labels = tokenizer(targets, max_length=max_target_length, truncation=True)["input_ids"]

        model_inputs["labels"] = labels
        examples[lcode]["input_ids"] = model_inputs["input_ids"]
        examples[lcode]["attention_mask"] = model_inputs["attention_mask"]
        examples[lcode]["labels"] = labels
    return examples



AttributeError: 'MTDistillationDatamodule' object has no attribute 'map'

In [2]:
print(batch["en-fr"]["source"].shape)
print(batch["en-ro"]["source"].shape)

torch.Size([4, 43])
torch.Size([1, 43])


In [3]:
from transformers import AutoModelForSeq2SeqLM

model_name = "t5-small"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [6]:
model(input_ids=batch["en-ro"]["source"], 
      attention_mask=batch["en-ro"]["attention_mask"], 
      decoder_input_ids=batch["en-ro"]["target"],     
      decoder_attention_mask=batch["en-ro"]["decoder_attention_mask"]
     ).logits

tensor([[[-21.9249,  -8.9075,  -9.3580,  ..., -46.9724, -47.1455, -47.0554],
         [-21.4823, -10.0551,  -4.1233,  ..., -42.9394, -43.0076, -42.9269],
         [-23.4109,  -9.6241, -13.6325,  ..., -47.0220, -47.0796, -47.0454],
         ...,
         [-10.6927,  -4.4714,  -9.4419,  ..., -39.3290, -39.4041, -39.4781],
         [-10.8941,  -4.4819,  -9.5339,  ..., -39.4958, -39.5699, -39.6404],
         [-10.9117,  -4.4439,  -9.5682,  ..., -39.4095, -39.4821, -39.5508]]],
       grad_fn=<UnsafeViewBackward0>)